In [31]:
import motmetrics as mm
import json

# Load ground truth and DeepSORT JSON files
with open('carla_.json', 'r') as f:
    gt_data = json.load(f)
with open('yolo.json', 'r') as f:
    ds_data = json.load(f)

# Create an accumulator that will be updated during each frame
acc = mm.MOTAccumulator(auto_id=True)

# Iterate through each frame of data
for frame in range(len(gt_data[0]['frames'])):
    # Get ground truth and DeepSORT data for the current frame
    gt_frame = gt_data[0]['frames'][frame]
    ds_frame = ds_data[0]['frames'][frame]


    gt_boxes = [[d['x'], d['y'], d['width'], d['height']] for d in gt_frame['annotations']]           
    ds_boxes = [[d['x'], d['y'], d['width'], d['height']] for d in ds_frame['hypotheses']]

    # Calculate the intersection over union (IOU) distance between ground truth and DeepSORT bounding boxes
    distances = mm.distances.iou_matrix(gt_boxes, ds_boxes, max_iou=0.5)

    # Update the accumulator with the calculated distances
    acc.update(
        [d['id'] for d in gt_frame['annotations']],
        [d['id'] for d in ds_frame['hypotheses']],
        distances
    )

# Calculate the MOTA and MOTP metrics from the accumulator data
mh = mm.metrics.create()
summary = mh.compute(acc, metrics=['mota', 'motp'], name='acc')
print(summary)

         mota      motp
acc -0.617647  0.413905
